In [1]:
import pandas as pd
import talib as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [2]:
# Load the 30-minute BTC/USDT data
data_30m = pd.read_csv('btcusdt_30m.csv', encoding='UTF-8')

# Ensure the columns are renamed correctly to match what the Backtesting library expects
data_30m.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# Parse datetime and set it as the index
data_30m['datetime'] = pd.to_datetime(data_30m['datetime'])
data_30m.set_index('datetime', inplace=True)


In [3]:
# Define the strategy with parameters for optimization
class EMASMACrossoverMACDStrategy(Strategy):
    # Define the parameters for EMA and SMA optimization
    ema_period = 45
    sma_period = 10
    macd_fastperiod = 12
    macd_slowperiod = 26
    macd_signalperiod = 9

    def init(self):
        # Calculate EMA and SMA
        self.ema = self.I(ta.EMA, self.data.Close, timeperiod=self.ema_period)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.sma_period)

        # Calculate MACD
        macd, signal, _ = ta.MACD(self.data.Close, fastperiod=self.macd_fastperiod, slowperiod=self.macd_slowperiod, signalperiod=self.macd_signalperiod)
        self.macd = self.I(lambda: macd)
        self.signal = self.I(lambda: signal)

    def next(self):
        # Buy condition: EMA crosses above SMA and MACD is above signal
        if crossover(self.ema, self.sma) and self.macd[-1] > self.signal[-1]:
            self.buy()

        # Sell condition: EMA crosses below SMA and MACD is below signal
        elif crossover(self.sma, self.ema) and self.macd[-1] < self.signal[-1]:
            self.sell()


In [4]:
# Run the backtest
bt = Backtest(data_30m, EMASMACrossoverMACDStrategy, cash=10000, commission=.002, exclusive_orders=True)
stats = bt.run()

# Print the results
print(stats)

C:\Users\Siddhant\AppData\Local\Temp\ipykernel_32212\3911277650.py:2: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data_30m, EMASMACrossoverMACDStrategy, cash=10000, commission=.002, exclusive_orders=True)


Start                     2018-01-01 05:30:00
End                       2022-01-12 05:30:00
Duration                   1472 days 00:00:00
Exposure Time [%]                   86.364863
Equity Final [$]                   56046.1843
Equity Peak [$]                   60704.49652
Return [%]                         460.461843
Buy & Hold Return [%]              216.860216
Return (Ann.) [%]                   53.279803
Volatility (Ann.) [%]               90.926121
Sharpe Ratio                         0.585968
Sortino Ratio                        1.525513
Calmar Ratio                         0.886472
Max. Drawdown [%]                  -60.103206
Avg. Drawdown [%]                   -2.676486
Max. Drawdown Duration      764 days 10:00:00
Avg. Drawdown Duration        7 days 06:25:00
# Trades                                  101
Win Rate [%]                        55.445545
Best Trade [%]                      42.809387
Worst Trade [%]                    -33.897439
Avg. Trade [%]                    